In [70]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [71]:
data = pd.read_csv('movie_bd_v5.csv', parse_dates=['release_date'])
data.sample()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
218,tt3007512,22500000,15520023,The Water Diviner,Russell Crowe|Olga Kurylenko|Jai Courtney|Isab...,Russell Crowe,A father's journey in search of his sons.,"Four years after the Battle of Gallipoli, Aust...",112,War|Drama,Hopscotch Features|RatPac Entertainment|Seven ...,2014-12-25,6.8,2014


Видно, что в столбцах cast, director, genres, production_company может быть больше одного наименования

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

In [5]:
# столбец прибыль фильма

data.insert(3, 'profit', data['revenue'] - data['budget'])

In [6]:
# вынесем жанры фильмов в отдельную таблицу, исходный столбец удалим

column_genres = data['genres'].str.split(pat='|').sort_values()

# находим все используемые жанры
all_genres = []
for this_record in column_genres:
    for this_genre in this_record:
        if not this_genre in all_genres:
            all_genres.append(this_genre)

# заполняем таблицу
genres = pd.DataFrame(data=False, index=range(data.shape[0]), columns=all_genres)
for i_row in range(data.shape[0]):
    for this_genre in column_genres.iloc[i_row]:
        genres.iloc[i_row][this_genre] = True

data.drop(columns='genres', inplace=True)

In [7]:
# вынесем режиссеров в отдельную таблицу, исходный столбец удалим

column_directors = data['director'].str.split(pat='|').sort_values()

# находим всех режиссеров
all_directors = []
for this_record in column_directors:
    for this_director in this_record:
        if not this_director in all_directors:
            all_directors.append(this_director)

# заполняем таблицу
directors = pd.DataFrame(data=False, index=range(data.shape[0]), columns=all_directors)
for i_row in range(data.shape[0]):
    for this_director in column_directors.iloc[i_row]:
        directors.iloc[i_row][this_director] = True

data.drop(columns='director', inplace=True)

In [8]:
# вынесем актеров в отдельную таблицу, исходный столбец удалим

column_actors = data['cast'].str.split(pat='|').sort_values()

# находим всех актеров
all_actors = []
for this_record in column_actors:
    for this_actor in this_record:
        if not this_actor in all_actors:
            all_actors.append(this_actor)

# заполняем таблицу
actors = pd.DataFrame(data=False, index=range(data.shape[0]), columns=all_actors)
for i_row in range(data.shape[0]):
    for this_actor in column_actors.iloc[i_row]:
        actors.iloc[i_row][this_actor] = True

data.drop(columns='cast', inplace=True)

In [9]:
# вынесем студии в отдельную таблицу, исходный столбец удалим

column_film_studios = data['production_companies'].str.split(pat='|').sort_values()

# находим всех актеров
all_film_studios = []
for this_record in column_film_studios:
    for this_film_studio in this_record:
        if not this_film_studio in all_film_studios:
            all_film_studios.append(this_film_studio)

# заполняем таблицу
film_studios = pd.DataFrame(data=False, index=range(data.shape[0]), columns=all_film_studios)
for i_row in range(data.shape[0]):
    for this_film_studio in column_film_studios.iloc[i_row]:
        film_studios.iloc[i_row][this_film_studio] = True

data.drop(columns='production_companies', inplace=True)

In [10]:
# месяц выпуска фильма

data['release_month'] = data['release_date'].map(lambda x : x.month)

# 1. У какого фильма из списка самый большой бюджет?


Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [11]:
# тут пишем ваш код для решения данного вопроса:
data[['budget', 'imdb_id', 'original_title']].sort_values(by='budget', ascending = False).head(1)

,budget,imdb_id,original_title
723,380000000,tt1298650,Pirates of the Caribbean: On Stranger Tides
1669,300000000,tt0449088,Pirates of the Caribbean: At World's End
14,280000000,tt2395427,Avengers: Age of Ultron
1014,260000000,tt0401729,John Carter
380,260000000,tt0398286,Tangled


In [12]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
# answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

# 2. Какой из фильмов самый длительный (в минутах)?

In [13]:
data[['runtime', 'imdb_id', 'original_title']].sort_values(by='runtime', ascending = False).head()

,runtime,imdb_id,original_title
1157,214,tt0279111,Gods and Generals
1081,201,tt0167260,The Lord of the Rings: The Return of the King
1736,191,tt0462322,Grindhouse
1336,187,tt0360717,King Kong
504,183,tt0213149,Pearl Harbor


In [14]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [15]:
data[['runtime', 'imdb_id', 'original_title']].sort_values(by='runtime', ascending = True).head()

,runtime,imdb_id,original_title
768,63,tt1449283,Winnie the Pooh
930,72,tt0280030,Return to Never Land
1097,72,tt0283426,The Jungle Book 2
1408,76,tt0361089,Valiant
1626,76,tt0299172,Home on the Range


In [16]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [17]:
round(data['runtime'].mean())

110

In [18]:
answers['4'] = '2. 110'

# 5. Каково медианное значение длительности фильмов? 

In [19]:
round(data['runtime'].median())

107

In [20]:
answers['5'] = '5. 107'

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [21]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data['profit'] = data['revenue'] - data['budget']
data[['profit', 'imdb_id', 'original_title']].sort_values(by='profit', ascending = False).head()

,profit,imdb_id,original_title
239,2544505847,tt0499549,Avatar
3,1868178225,tt2488496,Star Wars: The Force Awakens
0,1363528810,tt0369610,Jurassic World
4,1316249360,tt2820852,Furious 7
970,1299557910,tt0848228,The Avengers


In [22]:
answers['6'] = '5. Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [23]:
data[['profit', 'imdb_id', 'original_title']].sort_values(by='profit', ascending = True).head()

,profit,imdb_id,original_title
1245,-165710090,tt1210819,The Lone Ranger
1616,-119180039,tt0318974,The Alamo
809,-111007242,tt1305591,Mars Needs Moms
958,-92896027,tt0180052,The Adventures of Pluto Nash
1464,-84540684,tt0424095,Flushed Away


In [24]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [25]:
len(data[data['profit'] > 0])

1478

In [26]:
answers['8'] = '1. 1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [27]:
data_2008 = data[data['release_year'] == 2008]
data_2008[['profit', 'imdb_id', 'original_title']].sort_values(by='profit', ascending = False).head()

,profit,imdb_id,original_title
599,816921825,tt0468569,The Dark Knight
603,601636033,tt0367882,Indiana Jones and the Kingdom of the Crystal S...
607,557841637,tt0795421,Mamma Mia!
606,501744560,tt0441773,Kung Fu Panda
621,474029371,tt0448157,Hancock


In [28]:
answers['9'] = '4. The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [29]:
data_2012_2014 = data.query('release_year >= 2012 & release_year <= 2014')
data_2012_2014[['profit', 'imdb_id', 'original_title']].sort_values(by='profit', ascending = True).head()

,profit,imdb_id,original_title
1245,-165710090,tt1210819,The Lone Ranger
1214,-68351500,tt0790736,R.I.P.D.
1007,-51893525,tt1374992,Upside Down
1302,-51337973,tt0884726,Legends of Oz: Dorothy's Return
1235,-45510171,tt1308729,Bullet to the Head


In [30]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [31]:
genres.sum().sort_values(ascending=False).head()

Drama        782
Comedy       683
Thriller     596
Action       582
Adventure    415
dtype: int64

In [32]:
answers['11'] = '3. Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [33]:
genres[data['profit'] > 0].sum().sort_values(ascending=False).head()

Drama        560
Comedy       551
Thriller     446
Action       444
Adventure    337
dtype: int64

In [34]:
answers['12'] = '1. Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [35]:
directors_pivot = pd.DataFrame(data=0, index=directors.columns, columns=['total_revenue'])
for this_director in directors_pivot.index:
    directors_pivot['total_revenue'][this_director] = data[directors[this_director]]['revenue'].sum()
directors_pivot.sort_values(by='total_revenue', ascending=False).head()

,total_revenue
Peter Jackson,6490593685
Christopher Nolan,4167548502
David Yates,4154295625
Michael Bay,3886938960
J.J. Abrams,3579169916


In [36]:
answers['13'] = '5. Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [37]:
directors_pivot['action_count'] = 0
for this_director in directors_pivot.index:
    directors_pivot['action_count'][this_director] = genres[directors[this_director]]['Action'].sum()
directors_pivot.sort_values(by='action_count', ascending=False).head()

,total_revenue,action_count
Robert Rodriguez,930768969,9
Michael Bay,3886938960,7
Paul W.S. Anderson,1133874490,7
Antoine Fuqua,964376048,6
Ridley Scott,3264035909,6


In [38]:
answers['14'] = '3. Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [39]:
actors_pivot = pd.DataFrame(
    data=0, index=actors.columns, columns=['total_revenue_2012'])
for this_actor in actors_pivot.index:
    actors_pivot['total_revenue_2012'][this_actor] = data[(actors[this_actor]) & 
                                        (data['release_year'] == 2012)]['revenue'].sum()
actors_pivot.sort_values(by='total_revenue_2012', ascending=False).head()

,total_revenue_2012
Joseph Gordon-Levitt,5963675096
Josh Hartnett,2084446675
Seth Rogen,2040497780
Ben Kingsley,1916157910
Jude Law,1709210363


In [40]:
answers['15'] = '3. Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [41]:
mean_budget = data['budget'].mean()
actors_pivot['big_budget_count'] = 0
for this_actor in actors_pivot.index:
    actors_pivot['big_budget_count'][this_actor] = data[(
        actors[this_actor]) & (data['budget'] > mean_budget)]['imdb_id'].count()
actors_pivot.sort_values(by='big_budget_count', ascending=False).head()

,total_revenue_2012,big_budget_count
Mark Wahlberg,0,17
Bruce Willis,529000000,13
Russell Crowe,0,12
Adam Sandler,0,12
George Clooney,0,11


In [42]:
answers['16'] = '3. Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [43]:
genres[actors['Nicolas Cage']].sum().sort_values(ascending=False).head()

Drama       14
Thriller    11
Comedy      11
Action       6
Romance      6
dtype: int64

In [44]:
answers['17'] = '2. Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [45]:
data[film_studios['Paramount Pictures']][['profit', 'original_title', 'imdb_id']].sort_values(by='profit').head()

,profit,original_title,imdb_id
925,-64831034,K-19: The Widowmaker,tt0267626
1123,-60519261,Timeline,tt0300556
1722,-51788987,Next,tt0435705
1623,-46604061,Alfie,tt0375173
1631,-24805000,Twisted,tt0315297


In [46]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [47]:
pd.pivot_table(data, index='release_year', values='revenue', aggfunc=sum).sort_values(by='revenue', ascending=False).head()

,revenue
release_year,
2015,25449202382
2014,23405862953
2013,23213799791
2012,23079001687
2011,22676791872


In [48]:
answers['19'] = '5. 2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [49]:
wb_names = list(filter(lambda x : 'Warner Bros' in x, film_studios.columns))
film_studios[wb_names].max(axis=1).sum()

183

In [50]:
# если название начинается с Warner Bros и прибыль
wb_names = list(filter(lambda x : 'Warner Bros' in x, film_studios.columns))
data_wb = data[film_studios[wb_names].max(axis=1)]
pd.pivot_table(data_wb, index='release_year', values='profit', aggfunc=sum).sort_values(by='profit', ascending=False).head()

,profit
release_year,
2014,2295464519
2007,2201675217
2008,2134595031
2010,1974712985
2011,1871393682


In [51]:
# если название начинается с Warner и прибыль
wb_names = list(filter(lambda x : 'Warner' in x, film_studios.columns))
data_wb = data[film_studios[wb_names].max(axis=1)]
pd.pivot_table(data_wb, index='release_year', values='profit', aggfunc=sum).sort_values(by='profit', ascending=False).head()

,profit
release_year,
2014,2295464519
2007,2208217007
2008,2134595031
2010,1974712985
2011,1871393682


In [52]:
answers['20'] = '1. 2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [53]:
pivot_release_months = pd.pivot_table(data, index='release_month', values='imdb_id', aggfunc='count')
pivot_release_months.sort_values(by='imdb_id', ascending=False).head()

,imdb_id
release_month,
9,227
12,190
10,186
8,161
3,156


In [54]:
answers['21'] = '4. Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [55]:
summer_months = [6, 7, 8]
pivot_release_months.loc[summer_months].sum()

imdb_id    450
dtype: int64

In [56]:
answers['22'] = '2. 450'

# 23. Какой режиссер выпускает (суммарно по годам) больше всего фильмов зимой? 

In [57]:
winter_months = [12, 1, 2]
winter_movies = data[data['release_month'].isin(winter_months)][['original_title']]
target_directors = winter_movies.merge(directors, how='left', left_index=True, right_index=True)
target_directors.sum(numeric_only=True).sort_values(ascending=False).head()

Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
Shawn Levy           4
Adam Shankman        4
dtype: int64

In [58]:
answers['23'] = '5. Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [59]:
movies_title_length = data['original_title'].map(lambda x : len(x)).sort_values(ascending=False)
movies_title_length = pd.DataFrame(movies_title_length)
target_film_studios = movies_title_length.merge(film_studios, how='left', left_index=True, right_index=True)
for i_studio in range(target_film_studios.shape[1]):
    if target_film_studios.iloc[0, i_studio]:
        print(target_film_studios.columns[i_studio])

original_title
Twentieth Century Fox Film Corporation
Four By Two Productions


In [60]:
answers['24'] = '5. Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [61]:
overview_words_count = pd.Series(data=0, index=film_studios.columns, name='mean_words_count')
for this_studio in overview_words_count.index:
    overview_words_count[this_studio] = data[film_studios[this_studio]]['overview'].map(lambda x : len(x.split())).mean()
overview_words_count.sort_values(ascending=False).head()

Midnight Picture Show               175
Room 9 Entertainment                161
98 MPH Productions                  159
Heineken Branded Entertainment      159
Brookwell-McNamara Entertainment    156
Name: mean_words_count, dtype: int64

In [62]:
answers['25'] = '3. Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 

In [63]:
movies_rating = data[['original_title', 'vote_average']]
top_value = movies_rating.quantile(q=0.99)[0]
movies_rating[movies_rating['vote_average'] > top_value]

,original_title,vote_average
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
119,Guardians of the Galaxy,7.9
125,The Imitation Game,8.0
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
599,The Dark Knight,8.1
872,The Pianist,7.9


In [64]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [65]:
# количество пар актеров - оооооочень много
(len(actors.columns) * (len(actors.columns) - 1)) // 2

5798715

In [66]:
most_frequent_pair = {'name': 'nothing & nothing', 'count': 0}
for i_actor_1 in range(len(actors.columns)-1):
    actor_1 = actors.columns[i_actor_1]
    actor_1_column = actors[actor_1]
    for i_actor_2 in range(i_actor_1+1, len(actors.columns)):
        actor_2 = actors.columns[i_actor_2]
        pair_count = (actor_1_column & actors[actor_2]).sum()
        if pair_count > most_frequent_pair['count']:
            pair_name = '{0} & {1}'.format(actor_1, actor_2)
            print('found new pair - ' + pair_name)
            most_frequent_pair['name'] = pair_name
            most_frequent_pair['count'] = pair_count
        elif (pair_count == most_frequent_pair['count']) and (pair_count > 2):
            pair_name = '{0} & {1}'.format(actor_1, actor_2)
            print('same count - ' + pair_name)
print(most_frequent_pair)

found new pair - 50 Cent & Joy Bryant
found new pair - Delroy Lindo & Edgar RamÃ­rez
found new pair - Jennifer Aniston & Jason Bateman
same count - Bill Nighy & Johnny Depp
found new pair - Adam Sandler & Kevin James
found new pair - Johnny Depp & Helena Bonham Carter
same count - Ben Stiller & Owen Wilson
found new pair - Rupert Grint & Daniel Radcliffe
same count - Rupert Grint & Emma Watson
same count - Daniel Radcliffe & Emma Watson
{'name': 'Rupert Grint & Daniel Radcliffe', 'count': 8}


In [67]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

# Submission

In [68]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '5. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [69]:
# и убедиться что ни чего не пропустил)
len(answers)

27